In [2]:
import random
import math
import csv

# Activation functions
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class MLP:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        # Initialize weights with small random values
        self.weights_input_hidden = [[random.uniform(-1, 1) for _ in range(hidden_nodes)] for _ in range(input_nodes)]
        self.weights_hidden_output = [random.uniform(-1, 1) for _ in range(hidden_nodes)]
        
        # Initialize biases
        self.bias_hidden = [random.uniform(-1, 1) for _ in range(hidden_nodes)]
        self.bias_output = random.uniform(-1, 1)
    
    def forward(self, inputs):
        # Hidden layer calculations
        self.hidden_layer = []
        for i in range(self.hidden_nodes):
            weighted_sum = sum(inputs[j] * self.weights_input_hidden[j][i] for j in range(self.input_nodes)) + self.bias_hidden[i]
            self.hidden_layer.append(sigmoid(weighted_sum))
        
        # Output layer calculation
        output_weighted_sum = sum(self.hidden_layer[i] * self.weights_hidden_output[i] for i in range(self.hidden_nodes)) + self.bias_output
        self.output = sigmoid(output_weighted_sum)
        return self.output
    
    def backward(self, inputs, expected_output, learning_rate):
        # Calculate output layer error
        output_error = expected_output - self.output
        output_delta = output_error * sigmoid_derivative(self.output)
        
        # Calculate hidden layer errors
        hidden_errors = [self.weights_hidden_output[i] * output_delta for i in range(self.hidden_nodes)]
        hidden_deltas = [hidden_errors[i] * sigmoid_derivative(self.hidden_layer[i]) for i in range(self.hidden_nodes)]
        
        # Update weights and biases
        for i in range(self.hidden_nodes):
            self.weights_hidden_output[i] += learning_rate * output_delta * self.hidden_layer[i]
        self.bias_output += learning_rate * output_delta
        
        for i in range(self.input_nodes):
            for j in range(self.hidden_nodes):
                self.weights_input_hidden[i][j] += learning_rate * hidden_deltas[j] * inputs[i]
        for i in range(self.hidden_nodes):
            self.bias_hidden[i] += learning_rate * hidden_deltas[i]
    
    def train(self, dataset, epochs, learning_rate):
        for epoch in range(epochs):
            total_error = 0
            for data in dataset:
                inputs = [data[0], data[1]]
                expected_output = data[2]
                
                self.forward(inputs)
                self.backward(inputs, expected_output, learning_rate)
                
                total_error += (expected_output - self.output) ** 2
            
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Error: {total_error}")
    
    def predict(self, inputs):
        output = self.forward(inputs)
        return 1 if output > 0.5 else 0

# Load dataset from CSV
def load_dataset(filename):
    dataset = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            dataset.append([float(row[1]), float(row[2]), float(row[3])])
    return dataset

# Split dataset into training and validation sets
def split_dataset(dataset, train_ratio=0.8):
    random.shuffle(dataset)
    train_size = int(len(dataset) * train_ratio)
    return dataset[:train_size], dataset[train_size:]

# Load and split the dataset
dataset = load_dataset("xor.csv")
train_data, validation_data = split_dataset(dataset)

# Initialize and train the MLP
mlp = MLP(input_nodes=2, hidden_nodes=4, output_nodes=1)
mlp.train(train_data, epochs=10000, learning_rate=0.1)

# Testing the MLP on validation data
correct_predictions = 0
print("\nValidation Results:")
for data in validation_data:
    inputs = [data[0], data[1]]
    prediction = mlp.predict(inputs)
    actual_label = int(data[2])
    print(f"Input: {inputs}, Predicted Label: {prediction}, Actual Label: {actual_label}")
    if prediction == actual_label:
        correct_predictions += 1

# Calculate and print validation accuracy
accuracy = (correct_predictions / len(validation_data)) * 100
print(f"\nValidation Accuracy: {accuracy:.2f}%")

Epoch 0, Error: 203.96467966488396
Epoch 1000, Error: 10.703788106261044
Epoch 2000, Error: 8.563550743707953
Epoch 3000, Error: 7.711455251659272
Epoch 4000, Error: 7.204538242419433
Epoch 5000, Error: 6.853961746509258
Epoch 6000, Error: 6.591532487163072
Epoch 7000, Error: 6.385352092538099
Epoch 8000, Error: 6.21772548167116
Epoch 9000, Error: 6.077669252456801

Validation Results:
Input: [0.4287187330756249, 0.5856174890773638], Predicted Label: 1, Actual Label: 1
Input: [0.980305930913831, 0.5730905169394434], Predicted Label: 0, Actual Label: 0
Input: [0.8231804185071068, 0.12470051706005925], Predicted Label: 1, Actual Label: 1
Input: [0.3361808581463472, 0.36027277314075423], Predicted Label: 0, Actual Label: 0
Input: [0.8525184161505062, 0.9528055463471943], Predicted Label: 0, Actual Label: 0
Input: [0.0071557175961621255, 0.11351974824680489], Predicted Label: 0, Actual Label: 0
Input: [0.19134235993999116, 0.27442640921133543], Predicted Label: 0, Actual Label: 0
Input: [0